In [93]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
from datetime import date, datetime
from processing import Process

conf = SparkConf()
# conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.0')
# conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

procdata = Process()

In [10]:


geralprounicsv = pd.read_csv("pure_data/GERAL-cursos-prouni.csv")
addprounicsv = pd.read_csv("pure_data/pda-prouni.csv", sep=';')

geralprounicsv.to_xml("pure_data/geralprouni.xml")
addprounicsv.to_json("pure_data/prouni.json")

In [85]:
geralprouni = pd.DataFrame(pd.read_xml("pure_data/geralprouni.xml"))

quantidade_bolsas = geralprouni[['bolsa_integral_cotas', 'bolsa_integral_ampla', 'bolsa_parcial_cotas', 'bolsa_parcial_ampla']].sum(axis=1)
nota_bolsa = geralprouni[['nota_integral_ampla', 'nota_integral_cotas', 'nota_parcial_ampla', 'nota_parcial_cotas']].mean(axis=1)

prouni = pd.DataFrame({
    'mensalidade': geralprouni['mensalidade'], 
    'estado': geralprouni['uf_busca'], 
    'curso': geralprouni['nome'], 
    'quantidade_bolsas': quantidade_bolsas,
    'nota_bolsa': nota_bolsa
})
#geralprouni.head(5)
prouni.sample(n=10)

,mensalidade,estado,curso,quantidade_bolsas,nota_bolsa
40227,209.00,GO,Ciências Contábeis,1.0,585.10
2532,1417.46,PA,Direito,50.0,571.90
718,2048.40,RS,Engenharia Civil,5.0,571.48
2172,1473.37,BA,Psicologia,26.0,547.92
12401,713.28,SP,Gestão da Qualidade,9.0,528.98
28408,298.00,SP,Administração,1.0,500.24
27497,299.00,SP,Pedagogia,4.0,511.44
30558,279.00,RJ,Logística,2.0,591.25
20035,409.11,MS,Sistema de Informação,3.0,575.61
18927,443.20,GO,Administração,1.0,544.72


In [87]:
addprouni = pd.read_json("pure_data/prouni.json")

adicionalprouni = pd.DataFrame({
    'curso': addprouni['NOME_CURSO_BOLSA'],
    'estado': addprouni['SIGLA_UF_BENEFICIARIO_BOLSA'],
    'sexo': addprouni['SEXO_BENEFICIARIO_BOLSA'].map(lambda sexo: 'Masculino' if sexo == 'M' else 'Feminino'),
    'raca': addprouni['RACA_BENEFICIARIO_BOLSA'].map(lambda raca: 'Outro' if raca not in ['Parda', 'Branca', 'Preta'] else raca),
    'deficiente': addprouni['BENEFICIARIO_DEFICIENTE_FISICO'].map(lambda deficiente: 'Sim' if deficiente == 'S' else 'Não'),
    'idade': addprouni['DT_NASCIMENTO_BENEFICIARIO'].map(procdata.get_age_from_birthdate)
})
#addprouni.head(3)
adicionalprouni.sample(n=10)

,curso,estado,sexo,raca,deficiente,idade
132281,Curso Superior De Tecnologia Em Gestão Finance...,PB,Feminino,Parda,Não,23
69486,História,SC,Masculino,Branca,Não,27
145126,Engenharia De Produção - Ênfase Em Gestão E Lo...,PR,Masculino,Branca,Não,29
148803,Ciências Jurídicas E Sociais,RS,Feminino,Branca,Não,35
144012,Marketing,MT,Masculino,Parda,Não,32
215559,Engenharia Civil,RO,Feminino,Parda,Não,22
22403,Publicidade E Propaganda,SP,Feminino,Parda,Não,23
166520,Educação Física,AM,Masculino,Parda,Não,30
64662,Educação Física,SC,Feminino,Branca,Não,26
111926,Psicologia,MG,Feminino,Parda,Não,24


In [100]:
edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")

educacaobasica = pd.DataFrame({
    'estado': edbasica['SG_UF'],
    'esgoto_inexistente': edbasica['IN_ESGOTO_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'energia_inexistente': edbasica['IN_ENERGIA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'agua_inexistente': edbasica['IN_AGUA_INEXISTENTE'].map(procdata.get_label_from_floatbool),
    'acesso_internet': edbasica['IN_INTERNET'].map(procdata.get_label_from_floatbool),
    'faz_exame_selecao': edbasica['IN_EXAME_SELECAO'].map(procdata.get_label_from_floatbool),
    'especializada_deficientes': edbasica['IN_ESP'].map(procdata.get_label_from_floatbool),
    'ensino_tecnico': edbasica['IN_PROF_TEC'].map(procdata.get_label_from_floatbool)
})
#dfedbasica = spark.createDataFrame(geralprouni)
#edbasica.head(10)
educacaobasica.sample(10)

/tmp/ipykernel_12442/693207899.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  edbasica = pd.read_csv("pure_data/microdados_ed_basica_2021.csv", sep=";", encoding="latin-1")


,estado,esgoto_inexistente,energia_inexistente,agua_inexistente,acesso_internet,faz_exame_selecao,especializada_deficientes,ensino_tecnico
162696,SP,Não,Não,Não,Sim,Não,Sim,Não
14293,PA,Não,Não,Não,Sim,Não,Sim,Não
189361,PR,Não,Não,Não,Sim,Não,Sim,Não
129037,MG,Não,Não,Não,Não,Não,Não,Não
56514,CE,Não,Não,Não,Não,Não,Não,Não
112159,MG,Não,Não,Não,Sim,Não,Não,Não
19289,PA,Sim,Sim,Sim,Não,Não,Sim,Não
59350,RN,Não,Não,Não,Não,Não,Não,Não
203715,RS,Não,Não,Não,Sim,Não,Sim,Não
93812,BA,Não,Não,Não,Sim,Sim,Sim,Não


In [ ]:
adicionalprouni[(adicionalprouni['curso']=='Enfermagem') & (adicionalprouni['estado']=='SP')].sample(n=10)

In [14]:
from datetime import datetime
# dateparsed = datetime.strptime(date[:-5], "%Y-%m-%dT%H:%M:%S")

from difflib import SequenceMatcher
sm = SequenceMatcher(None, "S@o Pa*lo", "São Paulo")
sm.ratio()

0.7777777777777778